# <center>OA3801 - Comp Methods for O.R. II</center>
## <center>Charles Gallagher, David Wren, Cameron Holt</center>
### <center>Group project: An assessment of US county health service resilience </center>


### Situation

As a small team of analysts that have recently started looking into the capacity and resilience of medical services by county, we commenced researching US hospitals to identify counties that are under-serviced.

With the outbreak of COVID-19, focus has shifted towards resilience of US counties to deal with the virus


<img src="JHH.JPEG" width="800">
Source: https://www.businessinsider.com/top-us-hospitals-ranked-for-medical-care-by-us-news-2019-6


### References:
https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html#par_textimage_1383669527
https://openflights.org/data.html
https://raw.githubusercontent.com/CSSEGISandData/COVID19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv
https://systems.jhu.edu/research/public-health/ncov/


### Constraints, Limitations and Assumptions
1. Constraints
  1. Focus on US counties
  2. Incorporate information regarding reported cases of COVID-19 with hospital and demographic data
2. Limitations
  1. Short amount of time - realistic scope
  2. No SME input - limited information from online sources
  3. Johns Hopkins changed data reporting format two times during our work
3. Assumptions
  1. Johns Hopkins outbreak data is the most accurate data source available
  2. 2% transmission rate, 15% death rate for elderly (over 60) and 14 day recovery rate
  3. Demographic data has not changed significantly from 2017 information/forecasts
  4. All hospital types other than "Psychiatric" are able to treat COVID-19 patients
  5. Hospitals without any beds are not able to treat COVID-19 patients
  6. Cruise ship passangers not contagious as they are under quarantine then released upon medical clearance


### Motivation
Concern as to whether county health systems could cope with a sudden increase in throughput

<img src="StLouis.jpg" width="800">


### Tasks

1. Identify risk areas by state counties around the country based on:
  1. Proximity to reported cases
  2. Capacity of appropriate hospitals
  3. Aged and infant populations
  4. High unemployment areas
2. Assess the ability of the respective county hospitals to deal with the virus within respective counties
3. Identify high risk counties that warrant further resources to investigate and risk manage
4. Conduct a preliminary assessment of connectedness (via air)

### 3. Tools / Workflow

<img src="Workflow.PNG" width="800">

<img src="Equations.PNG" width="800">

In [1]:
# CODE

import pandas as pd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import re
import math 
import us
from scipy.integrate import odeint
import json
import requests
import branca
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import statsmodels.api as sm
# import gmplot # Use this to start plotting locations and graphics

#df2=pd.read_csv("Hospitals.csv",encoding="iso-8859-1",usecols=['LATITUDE','LONGITUDE','NAME','CITY','STATE','ZIP','BEDS','TYPE'])

#crono data
data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
US_data=data.loc[data['Country/Region']=='US']

# Zip code data need to match data to counties
zip2 = pd.read_csv("zip_code_database.csv")
zip_code = pd.read_csv("zip_code_database.csv", usecols = ['zip','county'])
zip_code = zip_code.rename(str.upper, axis='columns')

#County data needed to get center point locations of counties
county = pd.read_csv("2019_Gaz_counties_national.txt", sep='\t',usecols=[0,1,3,8,9 ])
county = county.rename(columns={"USPS": "STATE", "NAME": "COUNTY" })
county= county.rename(columns={'INTPTLAT':"LAT_C",'INTPTLONG':"LONG_C"})
county.rename(columns={ county.columns[4]: "LAT_C" }, inplace = True)

# get number of beds for each county
#hosp=pd.merge(df2, zip_code, on = 'ZIP', how= 'left')
#hosp=pd.merge(hosp, county, on = ['STATE','COUNTY'], how= 'left')
#hosp=hosp[hosp['BEDS'] >= 1] #The CAH designation is designed to reduce the financial vulnerability of rural hospitals and improve access to healthcare by keeping essential services in rural communities. To accomplish this goal, CAHs receive certain benefits, such as cost-based reimbursement for Medicare services
#county_beds = hosp[hosp['TYPE'] != 'PSYCHIATRIC'] # remove Other type of hospitals seem fine for the purpose of this study
#county_beds = hosp[["GEOID","BEDS"]]
#county_beds=county_beds.groupby("GEOID").sum().reset_index()

#wren_data=pd.read_csv('US_data-Wren.csv')

#df2=pd.read_csv("Hospitals.csv",encoding="iso-8859-1",
              #  usecols=['LATITUDE','LONGITUDE',
                        # 'NAME','CITY','STATE','ZIP',
                        # 'BEDS','TYPE'])

#crono data
data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
US_data=data.loc[data['Country/Region']=='US']


# Zip code data need to match data to counties 
zip2 = pd.read_csv("zip_code_database.csv")
zip_code = pd.read_csv("zip_code_database.csv", usecols = ['zip','county'])
zip_code = zip_code.rename(str.upper, axis='columns')

#County data needed to get center point locations of counties
county = pd.read_csv("2019_Gaz_counties_national.txt", sep='\t',usecols=[0,1,3,8,9 ])
county = county.rename(columns={"USPS": "STATE", "NAME": "COUNTY" })
county= county.rename(columns={'INTPTLAT':"LAT_C",'INTPTLONG':"LONG_C"})
county.rename(columns={ county.columns[4]: "LONG_C" }, inplace = True)

# get number of beds for each county
#hosp=pd.merge(df2, zip_code, on = 'ZIP', how= 'left')
#hosp=pd.merge(hosp, county, on = ['STATE','COUNTY'], how= 'left')
#hosp=hosp[hosp['BEDS'] >= 1] #The CAH designation is designed to reduce the financial vulnerability of rural hospitals and improve access to healthcare by keeping essential services in rural communities. To accomplish this goal, CAHs receive certain benefits, such as cost-based reimbursement for Medicare services
#county_beds = hosp[hosp['TYPE'] != 'PSYCHIATRIC'] # remove Other type of hospitals seem fine for the purpose of this study 
#county_beds = hosp[["GEOID","BEDS"]]
#ounty_beds=county_beds.groupby("GEOID").sum().reset_index()

#temp data get rid of when dave is done

US_data_long2 =pd.read_csv("us_data.csv")

##
master_data = pd.read_csv('US_data-Wren.csv')
#master_data =master_data.rename(columns={'Province/State':'County'})
master_data2 = pd.read_csv('master_df.csv')


##

master_data_risk=master_data2[['County ID','Risk Factor']]
master_data_risk=master_data_risk.rename(columns={'County ID':"GEOID"}).fillna(0)

risk_score = master_data_risk.set_index('GEOID')['Risk Factor']
#len(set(risk_score.index.unique()))



url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
county_data = f'{url}/us_county_data.csv'
county_geo = f'{url}/us_counties_20m_topo.json'



colorscale = branca.colormap.linear.YlGnBu_09.scale(0, 500)
colorscale2 = branca.colormap.linear.YlOrRd_09.scale(-1.1, 2.888)

colorscale2.caption = 'Risk Score'


risk_score = master_data_risk.set_index('GEOID')['Risk Factor']




# master dataframe risk score
def style_function2(feature):

    riskscore = risk_score.get(int(feature['id'][-5:]),None)

    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#FFFFFF' if riskscore is None else colorscale2(riskscore)
    }



m = folium.Map(
    location=[31.51073, -96.4247],
    tiles='cartodbpositron',
    zoom_start=4.4
)


# MAP 1############################################################
# master dataframe risk score
folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function2,
    name='County Risk Score'

).add_to(m)


    
for i in range(0,len(master_data)):
    folium.Circle(
          location=[master_data.iloc[i]['Lat'], master_data.iloc[i]['Long']],
          popup=('{}<br><br>Cases:{}'.format(master_data.iloc[i]['County'],int(master_data.iloc[i][[-10]]))), # county and lastest number of cases 
          radius=int(master_data.iloc[i][-10]),# lastest number of cases
          color='red',
          fill=True,
          fill_color='red'
       ).add_to(m)

# convert to (n, 2) nd-array format for heatmap
crono_heat = master_data[['Lat', 'Long']].as_matrix()

#plot heatmap
m.add_child(plugins.HeatMap(crono_heat, radius=(int(master_data.iloc[i][[-10]])*3),name='Crono Heat Map'))

m.add_child(colorscale2)



folium.LayerControl(name = 'Coronavirus disease (COVID-19)', show = None).add_to(m)




m.save('heat_map.html')
# MAP 1  End.   ############################################################


# MAp 2  Start   ############################################################


master_data_3 = master_data2.assign(pov_ratio=master_data2['Poverty_Count']/ master_data2['Total Population'])
master_data_3 = master_data_3.assign(age_young_ratio=master_data2['Age 0-14 Years']/ master_data2['Total Population'])
master_data_3 = master_data_3.assign(age_old_ratio=master_data2['Age 65 and Above']/ master_data2['Total Population'])
master_data_3 = master_data_3.assign(beds_ratio=master_data2['BEDS']/ master_data2['Total Population'])



# factor map

pov_ratio = master_data_3.set_index('County ID')['pov_ratio']
age_young_ratio = master_data_3.set_index('County ID')['age_young_ratio']
age_old_ratio = master_data_3.set_index('County ID')['age_old_ratio']
beds_ratio = master_data_3.set_index('County ID')['beds_ratio']
Unemployment_Rate =  master_data_3.set_index('County ID')['Unemployment_Rate']
pop_den = master_data_3.set_index('County ID')['Density per square mile of land area - Population']
risk_score = master_data_risk.set_index('GEOID')['Risk Factor']

colorscale = branca.colormap.linear.YlGnBu_09.scale(0,.3) #ratio
colorscale1 = branca.colormap.linear.YlGnBu_09.scale(0,.01) #hosp ratio
colorscale2a = branca.colormap.linear.YlGnBu_09.scale(0,18) # unemployment rate
colorscale3 = branca.colormap.linear.YlGnBu_09.scale(0,1000) #pop density 
colorscale4 = branca.colormap.linear.YlOrRd_09.scale(-1.1, 2.88)# risk score

# pov ratio
def style_function4(feature):

    score_pov = pov_ratio.get(int(feature['id'][-5:]),None)

    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#FFFFFF' if score_pov is None else colorscale(score_pov)
    }


# age_young_ratio
def style_function5(feature):

    score_young = age_young_ratio.get(int(feature['id'][-5:]),None)

    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#FFFFFF' if score_young is None else colorscale(score_young)
    }

# age_old_ratio
def style_function6(feature):

    score_old = age_old_ratio.get(int(feature['id'][-5:]),None)

    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#FFFFFF' if score_old is None else colorscale(score_old)
    }


# beds_ratio
def style_function7(feature):

    score_beds = beds_ratio.get(int(feature['id'][-5:]),None)

    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#FFFFFF' if score_beds is None else colorscale1(score_beds)
    }

# Unemployment
def style_function8(feature):

    Unemployment = Unemployment_Rate.get(int(feature['id'][-5:]),None)

    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#FFFFFF' if Unemployment is None else colorscale2a(Unemployment)
    }

# pop den
def style_function9(feature):

    pop = pop_den.get(int(feature['id'][-5:]),None)

    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#FFFFFF' if pop is None else colorscale3(pop)
    }

# master dataframe risk score
def style_function2a(feature):

    riskscore = risk_score.get(int(feature['id'][-5:]),None)

    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#FFFFFF' if riskscore is None else colorscale4(riskscore)
    }



#overall map instance
m2 = folium.Map(
    location=[31.51073, -96.4247],
    tiles='cartodbpositron',
    zoom_start=4.4
)



# pov ratio
folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function4,
    name='County Poverty Ratio'

).add_to(m2)

# age young raito
folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function5,
    name='County Age Under 14 Ratio'

).add_to(m2)

# age old raito
folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function6,
    name='County Age over 65 Ratio'

).add_to(m2)
    
    
# beds ratio
folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function7,
    name='Hospital Beds Ratio'

).add_to(m2)
    

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function8,
    name='Unemployment Rate'
).add_to(m2)

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function9,
    name='Population Density'
).add_to(m2)

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function2a,
    name='County Risk Score'

).add_to(m2)
    
    


folium.LayerControl().add_to(m2)

m2.save('factor_analysis.html')

# MAp 2  end   ############################################################

 
# plot 1 and 2   start   ############################################################

bar_data_high=master_data_3.sort_values(by = 'Risk Factor',  ascending=False).head(15).round(3).sort_values(by='Risk Factor', ascending = True)
bar_data_low=master_data_3.sort_values(by = 'Risk Factor',  ascending=True).head(15).round(3).sort_values(by='Risk Factor', ascending = True)

# Import the necessaries libraries
import plotly.offline as pyo
import plotly.graph_objs as go
# Set notebook mode to work in offline
pyo.init_notebook_mode()
# Create traces
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
fig1 = go.Figure()
#data_canada = px.data.gapminder().query("country == 'Canada'")
fig1 = px.bar(bar_data_high, y='COUNTY', x='Risk Factor',orientation='h',
            hover_data=['Risk Factor', 'STATE', 'pov_ratio','age_young_ratio',
                        'age_old_ratio','beds_ratio','Unemployment_Rate','Density per square mile of land area - Population'], color = 'Risk Factor',
               color_continuous_scale=["#160583", "#F49B00"],
            labels={'COUNTY':'County','STATE':'State', 'pov_ratio': "Poverty Ratio", 'age_young_ratio': 'Youth Ratio',
                   'age_old_ratio': 'Elderly Ratio', 'beds_ratio':'Hosptial Beds Ratio', 'Unemployment_Rate':'Unemployment Rate','Density per square mile of land area - Population':'Population Density'})
fig1.update_layout(title_text=' Counties with the Highest Risk Factor for COVID-19 ')

fig2 = go.Figure()
#data_canada = px.data.gapminder().query("country == 'Canada'")
fig2 = px.bar(bar_data_low, y='COUNTY', x='Risk Factor',orientation='h',
            hover_data=['Risk Factor', 'STATE', 'pov_ratio','age_young_ratio',
                        'age_old_ratio','beds_ratio','Unemployment_Rate','Density per square mile of land area - Population'], color = 'Risk Factor',
            labels={'COUNTY':'County','STATE':'State', 'pov_ratio': "Poverty Ratio", 'age_young_ratio': 'Youth Ratio',
                   'age_old_ratio': 'Elderly Ratio', 'beds_ratio':'Hosptial Beds Ratio', 'Unemployment_Rate':'Unemployment Rate','Density per square mile of land area - Population':'Population Density'},color_continuous_scale=['blue','green'])
fig3 =fig2.update_layout(title_text=' Counties with the Lowest Risk Factor for COVID-19 ')

############################################################## Gallagher updated moving heat maps ################


import folium
import folium.plugins as plugins
import numpy as np
import webbrowser

from datetime import datetime, timedelta


crono_heat=crono_heat.tolist()

c_list= []

for i in range(master_data.iloc[:, 5:-9].shape[1]):
    c_list.append(list(crono_heat))

temp_df = master_data.iloc[:, 5:-9]

time_data_list =[]
for i in range(0,master_data.iloc[:, 5:-9].shape[1]):
    row =[]
    for j in range(0,master_data.iloc[:, 5:-9].shape[0]):
        row.append([c_list[i][j][0],c_list[i][j][1], temp_df.iloc[:,i].tolist()[j]/100])
    time_data_list.append(row)

time_data_list = time_data_list[::-1]

time_index = [
    (datetime(2020,1,20) + k * timedelta(1)).strftime('%Y-%m-%d') for
    k in range(0,master_data.iloc[:, 5:-9].shape[1])]


#m = folium.Map([48., 5.], tiles='stamentoner', zoom_start=6)
m3 = folium.Map(
    location=[31.51073, -96.4247],
    tiles='cartodbpositron',
    zoom_start=4.4
)

hm = plugins.HeatMapWithTime(
    time_data_list,
    index=time_index,
    auto_play=True,
    max_opacity=0.3,
    radius=15
    
)

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function2a,
    name='County Risk Score'

).add_to(m3)

hm.add_to(m3)


################################################## end moving heat map################







## end plots##########################

ModuleNotFoundError: No module named 'us'

In [ ]:
import us
from scipy.integrate import odeint
import json
import requests
import branca
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import statsmodels.api as sm

In [ ]:
pip install branca

In [ ]:
m3 # heatmapwithtime, folium , TopoJason,  Risk factor Socre,  folium seems to be more flexiable than plotly allowing 
# for more layers from multiple data sources. However, to go further you may need use leaflet Javascript, 
# or leaflet for R which goes is closer to Javascript version of leaflet.


In [ ]:
m

In [ ]:
m2

In [ ]:
fig1.show()

In [ ]:
fig2.show()

In [ ]:
hina_in={}
korea_in={}
italy_in={}
iran_in={}
imports=pd.read_csv("Import.csv")
china_imp=imports[imports['ORIGIN_COUNTRY']=='CN']
china_imp=china_imp.sort_values(by='PASSENGERS',ascending=False)
c=china_imp['DEST'].unique()
iran=imports[imports['ORIGIN_COUNTRY']=='IR']
iran_imp=iran.sort_values(by='PASSENGERS',ascending=False)
ir=iran_imp['DEST'].unique()
korea=imports[imports['ORIGIN_COUNTRY']=='KR']
korea_imp=korea.sort_values(by='PASSENGERS',ascending=False)
kor=korea_imp['DEST'].unique()
italy=imports[imports['ORIGIN_COUNTRY']=='IT']
italy_imp=italy.sort_values(by='PASSENGERS',ascending=False)
it=italy_imp['DEST'].unique()
c=c[0:10]
ir=ir[0:10]
kor=kor[0:10]
it=it[0:10]
for element in c:
    china_in[element]=(china_imp[china_imp['DEST']==element]['PASSENGERS'].max())
for element in ir:
    iran_in[element]=(iran_imp[iran_imp['DEST']==element]['PASSENGERS'].max())
for element in kor:
    korea_in[element]=(korea_imp[korea_imp['DEST']==element]['PASSENGERS'].max())
for element in it:
    italy_in[element]=(italy_imp[italy_imp['DEST']==element]['PASSENGERS'].max())
plt.bar(range(len(china_in)), list(china_in.values()), align='center')
plt.xticks(range(len(china_in)),list(china_in.keys()),size=35)
plt.yticks(size=20)
plt.grid()
plt.title("Incoming passengers from China",size=50)


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
china_in={}
korea_in={}
italy_in={}
iran_in={}
imports=pd.read_csv("Import.csv")
china_imp=imports[imports['ORIGIN_COUNTRY']=='CN']
china_imp=china_imp.sort_values(by='PASSENGERS',ascending=False)
c=china_imp['DEST'].unique()

iran=imports[imports['ORIGIN_COUNTRY']=='IR']
iran_imp=iran.sort_values(by='PASSENGERS',ascending=False)
ir=iran_imp['DEST'].unique()
korea=imports[imports['ORIGIN_COUNTRY']=='KR']
korea_imp=korea.sort_values(by='PASSENGERS',ascending=False)
kor=korea_imp['DEST'].unique()
italy=imports[imports['ORIGIN_COUNTRY']=='IT']
italy_imp=italy.sort_values(by='PASSENGERS',ascending=False)
it=italy_imp['DEST'].unique()
c=c[0:10]
ir=ir[0:10]
kor=kor[0:10]
it=it[0:10]

for element in c:
    china_in[element]=(china_imp[china_imp['DEST']==element]['PASSENGERS'].max())
for element in ir:
    iran_in[element]=(iran_imp[iran_imp['DEST']==element]['PASSENGERS'].max())
for element in kor:
    korea_in[element]=(korea_imp[korea_imp['DEST']==element]['PASSENGERS'].max())
for element in it:
    italy_in[element]=(italy_imp[italy_imp['DEST']==element]['PASSENGERS'].max())
 
    
    
flow=pd.read_csv("flow.csv")
origin=flow['ORIGIN_AIRPORT_ID']
dest=flow['DEST_AIRPORT_ID']
qty=flow['PASSENGERS']


fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      #label = tran,
      color = "blue"
    ),
    link = dict(
      source = origin[0:50],
      target = dest[0:50],
      value = qty[0:50]
  ))])

fig.update_layout(title_text="Passenger flow", font_size=20)
fig.show()


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

#trans=pd.read_csv("translate.csv")
#tran=list(trans["Description"])
flow=pd.read_csv("newflow.csv")
origin=flow['ORIGIN_AIRPORT_ID']
dest=flow['DEST_AIRPORT_ID']

b1=list(dest[origin==12478])
while 12478 in b1: b1.remove(12478)
while 12892 in b1: b1.remove(12892)
while 14771 in b1: b1.remove(14771)
b2=list(dest[origin==12892])
while 12478 in b2: b2.remove(12478)
while 12892 in b2: b2.remove(12892)
while 14771 in b2: b2.remove(14771)
b3=list(dest[origin==14771])
while 14771 in b3: b3.remove(14771)
while 12478 in b3: b3.remove(12478)
while 12892 in b3: b3.remove(12892)

out1=[12478,12892,14771,12478,12892,14771,12478,12892,14771]
vals1=[24,20,14,23,27,16,26,8,5]
# Las Vegas 20, Seattle 19, Denver 19, Cincinatti 18,
a1=len(b1)*[12478]
a2=len(b2)*[12892]
a3=len(b3)*[14771]
in1=[1,1,1,2,2,2,3,3,3]
a=in1+a1+a2+a3
b=out1+b1+b2+b3
c=vals1+len(a)*[1]
 
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 1),
      #label = ,
      color = "blue"
    ),
    link = dict(
      source = a,
      target = b,
      value = c
  ))])

fig.update_layout(title_text='Passenger Flow', font_size=20)

fig.show()

In [ ]:
import numpy as np
from scipy.integrate import odeint
# A rough calculation of recovery timeline per region - requires population, contact and recovery rate inputs
# Total population, N.
N = 1628701
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma = 0.3, 1/14
#NY county: beds 11509, pop 1628701, elderly 268902
from matplotlib.pyplot import figure



# Initial number of infected and recovered individuals, I0 and R0.
I0, R0 = 1, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0
# A grid of time points (in days)
t = np.linspace(0, 160, 160)
# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt
# Initial conditions vector
y0 = S0, I0, R0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

# Plot the data on three separate curves for S(t), I(t) and R(t)
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, axisbelow=True)
ax.plot(t, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
ax.plot(t, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
ax.plot(t, R/1000, 'g', alpha=0.5, lw=2, label='Recovered')
#ax.plot(t,11509)
ax.set_title('NY county Recovery timeline', size = 50)
ax.set_xlabel('Time /days', size = 35)
ax.set_ylabel('Number (1000s)', size = 35)
ax.set_ylim(0,2000)
ax.yaxis.set_tick_params(length=0, width=2, labelsize = 20)
ax.xaxis.set_tick_params(length=0, width=2, labelsize = 20)
ax.grid(b=True, which='major', c='w', lw=100, ls='-')
legend = ax.legend(prop={'size': 30})
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.rcParams["figure.figsize"] = [25,15]
plt.show()
print("NY county: beds 11509, pop 1628701, elderly 268902")

In [ ]:
plt.bar(range(len(china_in)), list(china_in.values()), align='center')
plt.xticks(range(len(china_in)), list(china_in.keys()))
plt.grid()
plt.title("Incoming passengers from China")
plt.bar(range(len(italy_in)), list(italy_in.values()), align='center')
plt.xticks(range(len(italy_in)), list(italy_in.keys()))
plt.grid()
plt.title("Incoming passengers from Italy")
plt.bar(range(len(korea_in)), list(korea_in.values()), align='center')
plt.xticks(range(len(korea_in)), list(korea_in.keys()))
plt.grid()
plt.title("Incoming passengers from Korea") 

### Conclusion

1. Data evolves over time - any model requires constant revision and updating
2. OR teams very capable, but require SME integration to produce meaningful results
3. Software development becomes complicated very quickly - requires a structured approach
